In [7]:
import numpy as np
import pandas as pd

from functions import preprocess, train_default_catboost

In [8]:
pd.read_csv('./data/train.csv')

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Общежитие,Наличие_Матери,Наличие_Отца,Страна_Родители,Опекунство,Село,Иностранец,КодФакультета,СрБаллАттестата,Статус
0,72716,16019,2015,Жен,ОО,Английский язык,1998-02-08 00:00:00.000,"МБОУ ""СОШ №59""","Алтайский край, Барнаул г",2014.0,...,0.0,0,0.0,Россия,0.0,0.0,0.0,41.0,4.294,3
1,63306,14895,2010,Муж,ЦН,NaN,1992-04-18 00:00:00.000,NaN,NaN,NaN,...,0.0,1,1.0,Россия,0.0,0.0,0.0,28.0,67.000,4
2,82469,20103,2017,Жен,ДН,Английский язык,1999-03-21 00:00:00.000,МБОУ Алтайская СОШ №5,"Алтайский край, Алтайский р-н, Алтайское с",2017.0,...,0.0,1,1.0,Россия,0.0,0.0,0.0,41.0,4.133,4
3,81661,19838,2017,Жен,БН,Английский язык,1994-12-08 00:00:00.000,ФГБОУ ВО Алтайский государственный университет,"Алтайский край, г. Барнаул",2017.0,...,0.0,1,1.0,Россия,0.0,0.0,0.0,35.0,74.000,4
4,81509,19803,2017,Жен,БН,Английский язык,1995-06-28 00:00:00.000,ФГБОУ ВО Алтайский государственный университет,"Алтайский край, г. Барнаул",2017.0,...,0.0,1,1.0,Россия,0.0,0.0,0.0,28.0,53.000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13579,74283,17593,2016,Муж,ОО,Английский язык,1998-09-17 00:00:00.000,МБОУ СОШ №98,"Россия, Алтайский край, г Барнаул",2016.0,...,0.0,1,1.0,Россия,0.0,0.0,0.0,34.0,59.000,3
13580,71516,19433,2015,Жен,ОО,Немецкий язык,1997-09-19 00:00:00.000,"МКОУ ""Георгиевская СОШ""","Алтайский край, Ребрихинский р-н, Георгиевка с",2015.0,...,1.0,0,0.0,Россия,0.0,0.0,0.0,26.0,82.000,4
13581,76970,18723,2016,Муж,СН,Английский язык,1998-03-20 00:00:00.000,МБОУ СОШ №113 им. Сергея Семенова,"Россия, Алтайский край, г Барнаул",2016.0,...,0.0,1,1.0,Россия,0.0,0.0,0.0,41.0,3.400,4
13582,48548,14663,2012,Жен,ОО,NaN,1995-07-06 00:00:00.000,NaN,NaN,NaN,...,0.0,1,1.0,Россия,0.0,0.0,0.0,24.0,95.000,4


In [2]:
df_train, df_test = preprocess('./data/train.csv'), preprocess('./data/test.csv')

In [3]:
df_train.head()

,Код_группы,Год_Поступления,Основания,Общежитие,Наличие_Матери,Наличие_Отца,КодФакультета,СрБаллАттестата,Статус,Код1,Код2,Код3,Муж,Изучал_Англ,Год_Рождения,Возраст_Поступления,Перерыв,Учеба
ID,,,,,,,,,,,,,,,,,,
72716,16019,2015,оо,0,0,0,41,85.88,3,16,01,019,0,1,1998,17,1,ш
63306,14895,2010,цн,0,1,1,28,67.00,4,14,89,895,1,0,1992,18,0,н
82469,20103,2017,дн,0,1,1,41,82.66,4,20,10,103,0,1,1999,18,0,ш
81661,19838,2017,бн,0,1,1,35,74.00,4,19,83,838,0,1,1994,23,0,у
81509,19803,2017,бн,0,1,1,28,53.00,4,19,80,803,0,1,1995,22,0,у


In [4]:
df_train.dtypes

Код_группы             category
Год_Поступления           int64
Основания              category
Общежитие                 int32
Наличие_Матери            int32
Наличие_Отца              int32
КодФакультета          category
СрБаллАттестата         float64
Статус                    int64
Код1                   category
Код2                   category
Код3                   category
Муж                       int32
Изучал_Англ               int32
Год_Рождения              int64
Возраст_Поступления       int64
Перерыв                   int32
Учеба                  category
dtype: object

In [5]:
X, y = df_train.drop(['Статус'], axis=1, errors='ignore'), df_train['Статус'].replace({-1: 0, 3: 1, 4: 2})
X_test = df_test.drop(['Статус'], axis=1, errors='ignore')

In [6]:
clf = train_default_catboost(X, y, X_test, 'results/catboost_default.csv')

s:\miniconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Train len: 10867, val len: 2717, test len: 6691
F1 train: 0.8759500963665086 F1 test: 0.7677169132289018
{'Код_группы': 16.273765947810446, 'Год_Поступления': 4.76123296719679, 'Основания': 9.347906706944299, 'Общежитие': 0.5240197193012693, 'Наличие_Матери': 0.615696522185437, 'Наличие_Отца': 0.22510671086574047, 'КодФакультета': 9.445701920401905, 'СрБаллАттестата': 6.235497769671729, 'Код1': 17.098952291719442, 'Код2': 14.028779921687871, 'Код3': 6.2962845002039085, 'Муж': 1.3043981246120158, 'Изучал_Англ': 0.5710056801883785, 'Год_Рождения': 3.0612050798032375, 'Возраст_Поступления': 2.6585952642529507, 'Перерыв': 1.7361191079559462, 'Учеба': 5.815731765198757}
